## Data_Load & Split

In [1]:
import pandas as pd
import numpy as np
path = '/Users/ppangppang/Desktop/open/'

trn = pd.read_csv(path+'train.csv').drop(columns=['ID'])
val_ = pd.read_csv(path+'val.csv').drop(columns=['ID'])
val = val_.iloc[:,:-1]
lab = val_.iloc[:,-1]

tst = pd.read_csv(path+'test.csv')
tst_id = tst[['ID']]
tst = tst.drop(columns=['ID'])

from sklearn.preprocessing import StandardScaler
std = StandardScaler()
trn.iloc[:,:] = std.fit_transform(trn)
val.iloc[:,:] = std.fit_transform(val)
tst.iloc[:,:] = std.fit_transform(tst)

## baseline 참고
contamination = 0.0010551491277433877

nor = val_[val_['Class']==0]
abn = val_[val_['Class']==1]
X_trn = nor.iloc[:26000,:-1]
X_tst = pd.concat([nor.iloc[26000:,:-1], abn.iloc[:,:-1]])
y_tst = pd.concat([nor.iloc[26000:,-1],abn.iloc[:,-1]])

## Pyod Anomaly Detection package

In [2]:
# !pip install pyod
# git clone https://github.com/yzhao062/pyod.git

In [2]:
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, f1_score
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.suod import SUOD

ecod = ECOD(contamination=contamination)
copod = COPOD(contamination=contamination)
suod = SUOD(contamination=contamination)

ecod.fit(trn)
copod.fit(trn)
suod.fit(trn)

ecod_pred = ecod.predict(val)
copod_pred = copod.predict(val)
suod_pred = suod.predict(val)

print('='*30)
print('ecod_validation_confusion_matrix\n',confusion_matrix(lab, ecod_pred,labels=[1,0]))
print('ecod_validation_f1_score\n', f1_score(lab, ecod_pred,labels=[1,0]))
print('='*30)
print('copod_validation_confusion_matrix\n',confusion_matrix(lab, copod_pred,labels=[1,0]))
print('copod_validation_f1_score\n', f1_score(lab, copod_pred,labels=[1,0]))
print('='*30)
print('suod_validation_confusion_matrix\n',confusion_matrix(lab, suod_pred,labels=[1,0]))
print('suod_validation_f1_score\n', f1_score(lab, suod_pred,labels=[1,0]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.9min finished


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ecod_validation_confusion_matrix
 [[   13    17]
 [   20 28412]]
ecod_validation_f1_score
 0.4126984126984127
copod_validation_confusion_matrix
 [[   13    17]
 [   14 28418]]
copod_validation_f1_score
 0.456140350877193
suod_validation_confusion_matrix
 [[   12    18]
 [   15 28417]]
suod_validation_f1_score
 0.4210526315789474


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.7s finished


In [25]:
# suod : Anomaly detection ensemble model
suod.get_params()

{'approx_clf': None,
 'approx_clf_list': None,
 'approx_flag_global': True,
 'approx_ng_clf_list': None,
 'base_estimators': [LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
    metric_params=None, n_jobs=1, n_neighbors=15, novelty=True, p=2),
  LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
    metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2),
  HBOS(alpha=0.1, contamination=0.1, n_bins=10, tol=0.5),
  HBOS(alpha=0.1, contamination=0.1, n_bins=20, tol=0.5),
  COPOD(contamination=0.1, n_jobs=1),
  IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
      max_samples='auto', n_estimators=50, n_jobs=1, random_state=None,
      verbose=0),
  IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
      max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
      verbose=0),
  IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    

In [1]:
from sklearn.metrics import confusion_matrix, f1_score
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE

In [ ]:
ae = AutoEncoder(
    hidden_neurons=[128,64,32,16,16,32,64,128],
    validation_size=0.2,
    contamination=contamination,
    verbose=1,
    epochs=15
)
vae = VAE(
    encoder_neurons=[128,64,32,16],
    decoder_neurons=[16,32,64,128],
    validation_size=0.2,
    contamination=contamination,
    verbose=1,
    epochs=15
)

ae.fit(trn)
vae.fit(trn)

ae_pred = ae.predict(val)
vae_pred = vae.predict(val)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 30)                930       
                                                                 
 dropout_18 (Dropout)        (None, 30)                0         
                                                                 
 dense_25 (Dense)            (None, 30)                930       
                                                                 
 dropout_19 (Dropout)        (None, 30)                0         
                                                                 
 dense_26 (Dense)            (None, 128)               3968      
                                                                 
 dropout_20 (Dropout)        (None, 128)               0         
                                                                 
 dense_27 (Dense)            (None, 64)               

In [ ]:
print('='*30)
print('ae_validation_confusion_matrix\n',confusion_matrix(lab, ae_pred,labels=[1,0]))
print('ae_validation_f1_score\n', f1_score(lab, ae_pred,labels=[1,0]))
print('='*30)
print('vae_validation_confusion_matrix\n',confusion_matrix(lab, vae_pred,labels=[1,0]))
print('vae_validation_f1_score\n', f1_score(lab, vae_pred,labels=[1,0]))

ae_validation_confusion_matrix
 [[   11    19]
 [   21 28411]]
ae_validation_f1_score
 0.3548387096774193
vae_validation_confusion_matrix
 [[   11    19]
 [   21 28411]]
vae_validation_f1_score
 0.3548387096774193


## Classification with SMOTE(Imbalanced Data Problem)

In [5]:
## val 증강 및 분류
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_new, y_new = sm.fit_resample(val, (lab))

In [6]:
from flaml import AutoML
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, shuffle=True, test_size=0.1)
# Initialize an AutoML instance
automl = AutoML()
# Specify automl goal and constraint
automl_settings = {
    "time_budget": 60,  # in seconds
    "metric": 'f1',
    "task": 'classification',
}

# Train with labeled input data
automl.fit(X_train=X_train, y_train=y_train,**automl_settings)

# Predict
print(automl.predict_proba(X_train))
# Print the best model
print(automl.model.estimator)

/Users/ppangppang/opt/anaconda3/envs/ppangppang/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[flaml.automl: 08-06 11:00:46] {2427} INFO - task = classification
[flaml.automl: 08-06 11:00:46] {2429} INFO - Data split method: stratified
[flaml.automl: 08-06 11:00:46] {2432} INFO - Evaluation method: holdout
[flaml.automl: 08-06 11:00:46] {2551} INFO - Minimizing error metric: 1-f1
[flaml.automl: 08-06 11:00:46] {2691} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 08-06 11:00:46] {2993} INFO - iteration 0, current learner lgbm
[flaml.automl: 08-06 11:00:46] {3126} INFO - Estimated sufficient time budget=1827s. Estimated necessary time budget=45s.
[flaml.automl: 08-06 11:00:46] {3173} INFO -  at 0.4s,	es

[[9.99929053e-01 7.09471444e-05]
 [9.99204771e-01 7.95228628e-04]
 [1.02983006e-04 9.99897017e-01]
 ...
 [2.77072853e-05 9.99972293e-01]
 [1.02983006e-04 9.99897017e-01]
 [1.00000000e+00 0.00000000e+00]]
RandomForestClassifier(max_features=0.11438007536381348, max_leaf_nodes=86,
                       n_estimators=5, n_jobs=-1)


In [10]:
## Train_dataset학습 최적 결과
# ExtraTreesClassifier(criterion='entropy', max_features=0.20877119291744006,
#                      max_leaf_nodes=247, n_estimators=9, n_jobs=-1)

In [7]:
from sklearn.metrics import f1_score, confusion_matrix
pred = automl.predict(X_test)
print(confusion_matrix(y_test, pred))
print(f1_score(y_test, pred))

[[2874    0]
 [   0 2813]]
1.0


In [8]:
test = automl.predict(tst)
tst_id['Class'] = test
tst_id.to_csv(path+'extratree.csv',index=False)